In [1]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 44.9 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Fou

In [3]:
import pandas as pd
df = pd.read_csv("tweets.csv")

In [4]:
df.head()

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


In [5]:
df['text'][4]

'"Lord Jesus, your love brings freedom and pardon. Fill me with your Holy Spirit and set my heart ablaze with your l… https://t.co/VlTznnPNi8'

In [6]:
df['text'][1]

'Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po…'

In [7]:
df = df.drop(columns = ['id','keyword','location'])

In [8]:
df['target'].value_counts()

0    9256
1    2114
Name: target, dtype: int64

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [10]:
#Text consumable by BERT
preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
#BERT itself
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['target'],stratify = df['target'])

In [13]:
text_input = tf.keras.layers.Input(shape = (),dtype = tf.string,name='text-layer')
pre_processed_text = preprocess(text_input)
output = encoder(pre_processed_text)
d_layer = tf.keras.layers.Dropout(0.1,name = 'dropoutLayer')(output['pooled_output'])
d_layer = tf.keras.layers.Dense(1,activation = 'sigmoid',name = 'output')(d_layer)

In [14]:
model = tf.keras.Model(inputs = [text_input],outputs = [d_layer])

In [15]:
#Fine tuning - You dont train already trained model!!!!!!!!!!!!!!!!
#Only using the model to work for our purpose - only last layer will be trained
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text-layer (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_word_ids': (None,    0         ['text-layer[0][0]']          
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_type_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [16]:
metrics = [
    tf.keras.metrics.BinaryAccuracy(name = 'acuracy'),
    tf.keras.metrics.Precision(name = 'precision'),
    tf.keras.metrics.Recall(name = 'recall')
]

In [17]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = metrics)

In [18]:
model.fit(X_train,y_train,epochs = 3)

Epoch 1/3
267/267 [==============================] - 106s 341ms/step - loss: 0.4451 - acuracy: 0.8133 - precision: 0.4737 - recall: 0.0397
Epoch 2/3
267/267 [==============================] - 91s 340ms/step - loss: 0.4100 - acuracy: 0.8196 - precision: 0.5697 - recall: 0.1211
Epoch 3/3
267/267 [==============================] - 91s 342ms/step - loss: 0.3927 - acuracy: 0.8260 - precision: 0.6029 - recall: 0.1868


In [19]:
model.evaluate(X_test,y_test)

89/89 [==============================] - 33s 333ms/step - loss: 0.3655 - acuracy: 0.8389 - precision: 0.7320 - recall: 0.2117


[0.36549749970436096,
 0.8389025926589966,
 0.7320261597633362,
 0.21172022819519043]